In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

In [ ]:
combined_df = pd.read_csv(r"C:\Users\20223868\OneDrive - TU Eindhoven\Documents\School\year 3\BEP\code\output_combined.csv")

In [ ]:
combined_df.columns.tolist()

In [ ]:
combined_df.rename(columns={'MeanHR_3_prev_diff_3': 'MeanHR_prev_diff_3'}, inplace=True)


In [ ]:
combined_df.columns.tolist()

In [ ]:
for base_var in ['RMSSD', 'SDNN', 'MeanNN',"SDRMSSD", 'LF/HF', 'MeanHR_prev_diff']:
    raw_std = combined_df[base_var].std()
    windowed_std = combined_df[f"{base_var}_3"].std()
    print(f"{base_var} - 10s STD: {raw_std:.2f}, 30s STD: {windowed_std:.2f}")


In [ ]:
for base_var in ['RMSSD', 'SDNN', 'MeanNN',"SDRMSSD", 'LF/HF', 'MeanHR_prev_diff']:
    raw_cv = combined_df[base_var].std() / combined_df[base_var].mean()
    windowed_cv = combined_df[f"{base_var}_3"].std() / combined_df[f"{base_var}_3"].mean()
    print(f"{base_var} - 10s CV: {raw_cv:.2f}, 30s CV: {windowed_cv:.2f}")


In [ ]:
metrics = ['RMSSD', 'SDNN', 'MeanNN',"SDRMSSD", 'LF/HF', 'MeanHR_prev_diff']

for metric in metrics:
    try:
        # Compute per-participant μ and σ for 10s window
        group_10s = combined_df.groupby("participant")[metric]
        mu_10s = group_10s.mean().mean()
        sigma_10s = group_10s.std().mean()
        snr_10s = mu_10s / sigma_10s

        # Compute per-participant μ and σ for 30s window
        group_30s = combined_df.groupby("participant")[f"{metric}_3"]
        mu_30s = group_30s.mean().mean()
        sigma_30s = group_30s.std().mean()
        snr_30s = mu_30s / sigma_30s

        print(f"{metric} SNR (μ/σ) → 10s: {snr_10s:.2f}, 30s: {snr_30s:.2f}")

    except KeyError as e:
        print(f"{metric}: Missing data for one of the time windows ({e})")


In [ ]:
'''
#filter per participant
import numpy as np
variablelist = ['MeanHR_prev_diff_3',
    'RMSSD_3_mean_diff_3',
 'MeanNN_3_mean_diff_3',
 'SDNN_3_mean_diff_3',
 'SDRMSSD_3_mean_diff_3',
 'LF/HF_3_mean_diff_3',]

filtered_dfs = []
for participant, group_df in combined_df.groupby('participant'):
    mask = pd.Series(True, index=group_df.index)

    for variable in variablelist:
        z_scores = (group_df[variable] - group_df[variable].mean()) / group_df[variable].std()
        mask = mask & (np.abs(z_scores) < 3)
    filtered_dfs.append(group_df[mask])

filtered_df = pd.concat(filtered_dfs, ignore_index=True)
'''

In [ ]:
'''
#filter and visualize in total
variablelist= ['MeanHR_prev_diff_3',
    'RMSSD_3_mean_diff_3',
 'MeanNN_3_mean_diff_3',
 'SDNN_3_mean_diff_3',
 'SDRMSSD_3_mean_diff_3',
 'LF/HF_3_mean_diff_3',]
combined_df = filtered_df
for variable in variablelist:
    z_scores = zscore(combined_df[variable])
    filtered_df = combined_df[(abs(z_scores) < 3)]
    plt.figure()
    sns.histplot(combined_df[variable], bins=30, kde=True).set_title(variable+"before")
    sns.histplot(filtered_df[variable], bins=30, kde=True).set_title(variable+"after")
plt.show()
'''

In [ ]:
filtered_df.to_csv(r"C:\Users\20223868\OneDrive - TU Eindhoven\Documents\School\year 3\BEP\code\output_combined_cleaned.csv", index=False)
